In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface-hub
!pip install -q safetensors

##Hugging Face'e giriş

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

##Model ve tokenizer'ı yükleme

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

##Veri Setinin Hazırlanması

In [ ]:
python_explain_data = [
    {
        "code": """def add(a, b):
          return a + b""",
        "explanation": "Bu fonksiyon iki parametre alır (a ve b) ve ikisini toplayarak sonucu döndürür."
    },
    {
        "code": """numbers = [1, 2, 3, 4, 5]
          total = 0
          for n in numbers:
           total += n
          print(total)""",
        "explanation": "Önce 1'den 5'e kadar sayılardan oluşan bir liste tanımlanıyor. 'total' değişkeni 0 olarak başlatılıyor. for döngüsüyle listedeki her sayı 'total' üzerine ekleniyor. Son olarak toplam ekrana yazdırılıyor."
    },
    {
        "code": """for i in range(5):
          print("Hello", i)""",
        "explanation": "range(5) ifadesi 0'dan 4'e kadar sayılar üretir. Döngü her adımda 'Hello' ve o anki 'i' değerini ekrana yazdırır."
    },
    {
        "code": """def is_even(n):
          if n % 2 == 0:
            return True
          else:
            return False""",
        "explanation": "Bu fonksiyon n sayısının 2 ile bölümünden kalanı kontrol eder. Kalan 0 ise sayı çifttir ve True döndürülür, değilse False döndürülür."
    },
    {
        "code": """try:
          x = int(input("Sayı gir: "))
          print(10 / x)
          except ZeroDivisionError:
          print("Sıfıra bölemezsin!")
""",
        "explanation": "Kullanıcıdan bir sayı alınıp tamsayıya çevrilir. Ardından 10/x hesaplanır. Eğer kullanıcı 0 girerse ZeroDivisionError hatası oluşur ve except bloğu çalışarak uyarı mesajı yazdırılır."
    },
]


##HuggingFace Dataset nesnesi oluşturma

In [ ]:
from datasets import Dataset

def make_example(example):
    instruction = (
        "Türkçe ve anlaşılır bir şekilde, aşağıdaki Python kodunu satır satır açıkla:\n"
        f"{example['code']}"
    )
    target = example["explanation"]
    return {"input_text": instruction, "target_text": target}

converted = [make_example(e) for e in python_explain_data] #Listedeki her elemanı birer örneğe dönüştürüyoruz
dataset = Dataset.from_list(converted) #Listeyi HuggingFace dataset formatında dönüştürüyoruz
dataset = dataset.train_test_split(test_size=0.2, seed=42) #Veri setini dağıtıyoruz train %80 test %20

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 4
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 1
    })
})


In [ ]:
dataset["train"][0]

{'input_text': 'Türkçe ve anlaşılır bir şekilde, aşağıdaki Python kodunu satır satır açıkla:\nfor i in range(5):\n          print("Hello", i)',
 'target_text': "range(5) ifadesi 0'dan 4'e kadar sayılar üretir. Döngü her adımda 'Hello' ve o anki 'i' değerini ekrana yazdırır."}

##Tokenization ayarları

In [ ]:
max_input_length =  256 #girdi
max_target_length = 128 #çıktı

def preprocess(batch):
    # Encoder
    model_inputs = tokenizer(
        batch["input_text"],
        max_length=max_input_length,
        truncation=True,        # max_input_length'i aşarsa kes
        padding="max_length",   # max_input_length'e kadar padding yap
    )

    # Decoder
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["target_text"],
            max_length=max_target_length,
            truncation=True,
            padding="max_length",
        )

    #doğru cevaplar temsil ediliyor
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Preprocess fonktiyonları uygulanır ve input/target kolonları kaldırılır

In [ ]:
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["input_text", "target_text"])

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
})

##Data collator ve eğitim argümanları

"evaluate" ekleyip metrik karşılaşması yapılabilir

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) #dinamik padding ile model beslenir

training_args = Seq2SeqTrainingArguments(
    output_dir="flan-t5-python-explainer",
    eval_strategy="steps",
    eval_steps=20,
    logging_steps=10,
    save_steps=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=100,             #Toplam epoch sayısı
    learning_rate=5e-5,               #Öğrenme oranı
    predict_with_generate=True,
    fp16=False,
    save_total_limit=1,
    load_best_model_at_end=False,
)

trainer = Seq2SeqTrainer(
    model=model,                                # Eğitilecek model
    args=training_args,                         # Eğitim ayarları
    train_dataset=tokenized_dataset["train"],   # Eğitim verisi
    eval_dataset=tokenized_dataset["test"],     # Test verisi
    tokenizer=tokenizer,                        # Tokenizer
    data_collator=data_collator,
)


/tmp/ipython-input-1522467864.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yenes8300 (yenes8300-amasya-niversitesi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
20,14.196000,6.419579
40,10.279500,5.401612
60,8.000300,4.586006
80,6.298300,3.923098
100,5.731400,3.831260


TrainOutput(global_step=100, training_loss=9.72016716003418, metrics={'train_runtime': 124.2267, 'train_samples_per_second': 3.22, 'train_steps_per_second': 0.805, 'total_flos': 37178100940800.0, 'train_loss': 9.72016716003418, 'epoch': 100.0})

Modeli kaydetme

In [ ]:
trainer.save_model("flan-t5-python-explainer")
tokenizer.save_pretrained("flan-t5-python-explainer")

('flan-t5-python-explainer/tokenizer_config.json',
 'flan-t5-python-explainer/special_tokens_map.json',
 'flan-t5-python-explainer/spiece.model',
 'flan-t5-python-explainer/added_tokens.json',
 'flan-t5-python-explainer/tokenizer.json')

##Yeni kodları açıklayan fonksiyon tanımlama

Bu fonksiyon, verilen Python kodu için Türkçe açıklama üretir.

In [ ]:
def explain_code(code: str, max_new_tokens: int = 128) -> str:
    instruction = (
        "Türkçe ve anlaşılır bir şekilde, aşağıdaki Python kodunu satır satır açıkla:\n"
        f"{code}"
    )

    # Instructionı token'lara çeviriyoruz.
    inputs = tokenizer(
        instruction,
        return_tensors="pt",          # PyTorch tensör formatında döndür
        truncation=True,
        max_length=max_input_length,
    ).to(model.device)                # Modelin çalıştığı cihaza (CPU/GPU) gönder

    #Modelin çıktısı
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True,          # Uygunsa modeli durdur
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

##Test

In [ ]:
test_code = """my_list = [10, 20, 30]
for i, val in enumerate(my_list):
    print(i, val)"""

print(explain_code(test_code))

Python kodunu satr akla: my_list = [10, 20, 30] for i, val in enumerate(my_list): print(i, val)


##HuggingFace

In [ ]:
from huggingface_hub import create_repo, upload_folder, HfApi

In [ ]:
api = HfApi()
me = api.whoami()
print(me)

{'type': 'user', 'id': '6762ad14900d40432204f487', 'name': 'Yenes', 'fullname': 'Yusuf Elmas', 'isPro': False, 'avatarUrl': '/avatars/0bf00558fb84f1c59a69e67ba8d88f1f.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'Deneme', 'role': 'fineGrained', 'createdAt': '2025-12-10T11:39:46.786Z', 'fineGrained': {'canReadGatedRepos': False, 'global': [], 'scoped': [{'entity': {'_id': '6762ad14900d40432204f487', 'type': 'user', 'name': 'Yenes'}, 'permissions': ['repo.content.read', 'repo.write']}]}}}}


In [ ]:
USERNAME = me["name"]
repo_id = f"{USERNAME}/flan-t5-python-explainer"

In [ ]:
create_repo(repo_id=repo_id, exist_ok=True)

RepoUrl('https://huggingface.co/Yenes/flan-t5-python-explainer', endpoint='https://huggingface.co', repo_type='model', repo_id='Yenes/flan-t5-python-explainer')

In [ ]:
upload_folder(
    repo_id=repo_id,
    folder_path="flan-t5-python-explainer",
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...eckpoint-100/spiece.model: 100%|##########|  792kB /  792kB            

  ...on-explainer/spiece.model: 100%|##########|  792kB /  792kB            

  ...ckpoint-100/rng_state.pth:  77%|#######7  | 11.3kB / 14.6kB            

...

  ...eckpoint-100/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

  ...int-100/training_args.bin:   8%|7         |   460B / 5.97kB            

  ...366813.d2917ba49a56.331.0:   8%|7         |   756B / 9.82kB            

  ...plainer/training_args.bin:   8%|7         |   460B / 5.97kB            

CommitInfo(commit_url='https://huggingface.co/Yenes/flan-t5-python-explainer/commit/0687dbe8e1462565ff09dd4f8aac5d720f7702e2', commit_message='Upload folder using huggingface_hub', commit_description='', oid='0687dbe8e1462565ff09dd4f8aac5d720f7702e2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Yenes/flan-t5-python-explainer', endpoint='https://huggingface.co', repo_type='model', repo_id='Yenes/flan-t5-python-explainer'), pr_revision=None, pr_num=None)